In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")
import re
import nltk
import dask.array as da

In [46]:
data = pd.read_csv('data/tags.csv')

In [47]:
data.head()

,name,tags
0,Благословение небожителей. Том 2,все в эт мир имет счет и удач и невезен бед пр...
1,Благословение небожителей. Том 1,в незапамятн врем си лянут быт наследн принц г...
2,Лето в пионерском галстуке,юр возвраща в пионерск лагер сво юност спуст д...
3,Электрошок. Внезапно,нов книг от автор культов цикл древн наслед и ...
4,Билли Саммерс,нов увлекательн рома от автор культов бестселл...


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(encoding='utf-8')

## Векторизация текста на основе модели "мешка слов"

Векторизация текста поддерживается библиотекой [scikit-learn](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)

### Использование класса [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

Подсчитывает количество слов словаря, входящих в данный текст.

In [48]:
cv.fit(data['tags'])
corpusVocab = cv.vocabulary_
print('Количество сформированных признаков - {}'.format(len(corpusVocab)))

Количество сформированных признаков - 40132


In [49]:
for i in list(corpusVocab)[1:10]:
    print('{}={}'.format(i, corpusVocab[i]))

мир=18083
имет=11807
счет=33625
удач=35436
невезен=19819
бед=1917
преследова=26493
си=31002
лян=16811


In [9]:
vectors = cv.fit_transform(data['tags'].values.astype('U'))

In [10]:
vectors

<9070x46222 sparse matrix of type '<class 'numpy.int64'>'
	with 602058 stored elements in Compressed Sparse Row format>

In [11]:
vectors.shape

(9070, 46222)

In [12]:
vectors.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [13]:
# Непустые значения нулевой строки
[i for i in vectors.todense()[0].getA1() if i>0][0:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [14]:
cv.get_feature_names()[23567:23587]

C:\Users\kenia\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['непродолжительный',
 'непрожитый',
 'непроизвольно',
 'непроизвольный',
 'непройти',
 'непролазный',
 'непроницаемый',
 'непростительный',
 'непросто',
 'непростой',
 'непротиворечивый',
 'непроходимый',
 'непроходить',
 'непрочность',
 'непрочный',
 'непрочь',
 'непрошенный',
 'непрошеный',
 'непрощённый',
 'нептун']

### CountVectorizer with spacy

In [3]:
data = pd.read_csv('data/spacylem.csv')
data.head()

,name,tags
0,Благословение небожителей. Том 2,ве мир имет счет удач невезен бед преследова с...
1,Благословение небожителей. Том 1,незапамятн врем се лян наследн принц государст...
2,Лето в пионерском галстуке,юр возвраща пионерск лагер сво юност спуст два...
3,Электрошок. Внезапно,нов книг автор культов цикл древн наслед кажд ...
4,Билли Саммерс,нов увлекательн рома автор культов бестселлер ...


In [5]:
cv.fit(data['tags'])
corpusVocab = cv.vocabulary_
print('Количество сформированных признаков - {}'.format(len(corpusVocab)))

Количество сформированных признаков - 42206


In [6]:
for i in list(corpusVocab)[1:10]:
    print('{}={}'.format(i, corpusVocab[i]))

мир=18862
имет=12423
счет=35449
удач=37344
невезен=20692
бед=1969
преследова=27851
се=32295
лян=17557


In [9]:
vectors = cv.fit_transform(data['tags'].values.astype('U'))
# Непустые значения нулевой строки
[i for i in vectors.todense()[0].getA1() if i>0][0:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [10]:
cv.get_feature_names()[29443:29463]

C:\Users\kenia\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['пышет',
 'пышк',
 'пышн',
 'пышногруд',
 'пышност',
 'пьедеста',
 'пьедр',
 'пьемонт',
 'пьер',
 'пьес',
 'пьеск',
 'пьет',
 'пьетр',
 'пьех',
 'пьюджет',
 'пьюз',
 'пьюзон',
 'пьюзоскандинавск',
 'пьян',
 'пьянеет']

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
%%time
similarity = cosine_similarity(vectors)

CPU times: total: 3.97 s
Wall time: 4.08 s


In [13]:
similarity

array([[1.        , 0.27707837, 0.10873971, ..., 0.01911099, 0.11770993,
        0.        ],
       [0.27707837, 1.        , 0.13650473, ..., 0.01399457, 0.16281553,
        0.07881104],
       [0.10873971, 0.13650473, 1.        , ..., 0.        , 0.18041543,
        0.08247861],
       ...,
       [0.01911099, 0.01399457, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.11770993, 0.16281553, 0.18041543, ..., 0.        , 1.        ,
        0.22568628],
       [0.        , 0.07881104, 0.08247861, ..., 0.        , 0.22568628,
        1.        ]])

In [14]:
similarity[0]

array([1.        , 0.27707837, 0.10873971, ..., 0.01911099, 0.11770993,
       0.        ])

In [69]:
type(similarity)

numpy.ndarray

In [70]:
list(zip(list(range(len(similarity[0].tolist()))),similarity[0].tolist()))

TypeError: 'Int64Index' object is not callable

In [72]:
a = [1,2,3,4,5,6,7,8,9]
a

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [73]:
enumerate(a)

In [74]:
list(enumerate(a))

TypeError: 'Int64Index' object is not callable

In [15]:
list(enumerate(similarity[0]))

[(0, 0.999999999999999),
 (1, 0.2770783726646877),
 (2, 0.10873970905021002),
 (3, 0.03771571432023571),
 (4, 0.05492184689546835),
 (5, 0.020236116462908824),
 (6, 0.09859996028394398),
 (7, 0.09163376157740079),
 (8, 0.025993762245501817),
 (9, 0.1044073795327749),
 (10, 0.039487359486949224),
 (11, 0.13453050062515784),
 (12, 0.06511170133930119),
 (13, 0.044651954892975174),
 (14, 0.034658349660669094),
 (15, 0.0637991802113209),
 (16, 0.032555850669650596),
 (17, 0.03822198817224581),
 (18, 0.1089289863197396),
 (19, 0.07804228110385687),
 (20, 0.021223817998900444),
 (21, 0.0645332619692431),
 (22, 0.05264981264926563),
 (23, 0.06898028273475494),
 (24, 0.04956815970966097),
 (25, 0.026324906324632816),
 (26, 0.03495083037059981),
 (27, 0.08024208205741783),
 (28, 0.1395541389950575),
 (29, 0.044743737014268294),
 (30, 0.021586644588817278),
 (31, 0.026669037353133248),
 (32, 0.04660273245009001),
 (33, 0.03485251560377424),
 (34, 0.08930390978595035),
 (35, 0.04429039465181195),

In [16]:
sorted(similarity[0],reverse=True)

[0.999999999999999,
 0.2770783726646877,
 0.223374237314982,
 0.19861656388262144,
 0.19843281603361565,
 0.19485809450486585,
 0.19451950503185497,
 0.19314479165731382,
 0.19062092313368,
 0.1894422005883515,
 0.1886705387227641,
 0.18842421687162592,
 0.18788455692040834,
 0.18531760113378795,
 0.1850731581187133,
 0.1827006660620956,
 0.18151372660830078,
 0.17937400083354377,
 0.1791496430211196,
 0.17818884212705496,
 0.17757120130114434,
 0.1757151459646611,
 0.1748235497692681,
 0.17437145811572893,
 0.17251115087387112,
 0.17236327454358857,
 0.17200698993472502,
 0.17181330295762648,
 0.1716446746239428,
 0.17155007848855514,
 0.17098528222033246,
 0.17084843923544982,
 0.17042291319677882,
 0.17029512324323617,
 0.170169118695282,
 0.17008506380317154,
 0.17008506380317154,
 0.17008506380317154,
 0.1700254554204575,
 0.1697207144410607,
 0.16937113484345634,
 0.16937113484345634,
 0.16908747453646442,
 0.16900092426997898,
 0.16885505178119048,
 0.167934853912572,
 0.1679348

In [32]:
def recommend(rusbook):
    rusbook_index = data[data['name'] == rusbook].index[0]
    distances = similarity[rusbook_index]
    rusbook_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:11]
    
    for i in rusbook_list:
       print(data.name[data.iloc[i[0]].name])

In [33]:
recommend('Благословение небожителей. Том 2 ')

Благословение небожителей. Том 1 
Золотое сердце Вавилона 
В Калифорнии морозов не бывает 
Забыть нельзя вспомнить 
Король говорит! История о преодолении, о долге и чести, о лидерстве, об иерархии и о настоящей дружбе 
Долорес Клейборн 
Дивергент 
Против ветра, мимо облаков 
Через год в это же время 
Кари Мора 


In [35]:
recommend('Тень и кость ')

Штурм и буря 
Нежно 
Шёпот магии 
Серафина 
Счастье рядом 
Эхо между нами 
Шестерка воронов 
Братья Карилло. Обретая надежду 
Зов темной крови 
Асфальт 


In [39]:
data = pd.read_csv('data/dataclean.csv')

In [37]:
import pickle

In [38]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [41]:
pickle.dump(data.to_dict(),open('books_dict.pkl','wb'))